In [2]:
pip install -U transformers accelerate bitsandbytes sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_6

In [1]:
!pip install accelerate

In [8]:
!pip install tqdm notebook==7.1.2 openai elasticsearch pandas scikit-learn ipywidgets

In [2]:
#Mistral 7b
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

In [1]:
import os
os.environ['HF_TOKEN'] = 'your token'

In [1]:
from huggingface_hub import login
login(token=os.environ['HF_TOKEN'])

Q1. Running Ollama with Docker

docker run -it \
    --rm \
    -v ollama:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollama
docker exec -it ollama bash
ollama -v

In [ ]:
docker exec -it ollama bash

Q2. Downloading an LLM
docker exec -it ollama bash
ollama pull gemma:2b
root@b1b14f2b4b1c:~/.ollama/models/manifests/registry.ollama.ai/library/gemma# cat 2b
{"schemaVersion":2,"mediaType":"application/vnd.docker.distribution.manifest.v2+json","config":{"mediaType":"application/vnd.docker.container.image.v1+json","digest":"sha256:887433b89a901c156f7e6944442f3c9e57f3c55d6ed52042cbb7303aea994290","size":483},"layers":[{"mediaType":"application/vnd.ollama.image.model","digest":"sha256:c1864a5eb19305c40519da12cc543519e48a0697ecd30e15d5ac228644957d12","size":1678447520},{"mediaType":"application/vnd.ollama.image.license","digest":"sha256:097a36493f718248845233af1d3fefe7a303f864fae13bc31a3a9704229378ca","size":8433},{"mediaType":"application/vnd.ollama.image.template","digest":"sha256:109037bec39c0becc8221222ae23557559bc594290945a2c4221ab4f303b8871","size":136},{"mediaType":"application/vnd.ollama.image.params","digest":"sha256:22a838ceb7fb22755a3b0ae9b4eadde629d19be1f651f73efb8c6b4e2cd0eea0","size":84}]}root@b1b14f2b4b1c:~/.ollama/models/manifests/registry.ollama.ai/library/gemma#
root@b1b14f2b4b1c:~/.ollama/models/manifests/registry.ollama.ai/library/gemma#

In [ ]:
#Q3. Running the LLM

In [ ]:
import requests
import minsearch
import pandas

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch
es = Elasticsearch("http://localhost:9200")
es.info()

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

In [ ]:
for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

In [10]:
from openai import OpenAI
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)
def llm_ollama(prompt):


    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [26]:

from openai import OpenAI
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)
def llm_ollama(prompt):


    response = client.chat.completions.create(
        model='gemma:2b',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [27]:

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm_ollama(prompt)
    return answer

In [28]:
rag("I just discovered the course. Can I still join it?")

'The passage does not mention whether or not the user can join the course after the start date, so I cannot answer this question from the provided context.'

In [25]:
rag("10 * 10")

' The answer provided seems to be discussing unrelated topics rather than solving or providing information about multiplying two numbers as in the question prompt "10 * 10". Therefore I will provide an independent response focusing solely on answering your original math problem, which is a straightforward multiplication of integers.\n\nANSWER: The result of calculating ten times ten (10 * 10) equals one hundred (100).'

In [ ]:
#docker exec -it ollama bash

#Q4
mkdir ollama_files

docker run -it \
    --rm \
    -v ./ollama_files:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollama

In [ ]:
mkdir ollama_files

docker run -it --rm -v ./ollama_files:/root/.ollama -p 11434:11434 --name ollama ollama/ollama

In [3]:
#Q5: Docker file
#download model

In [3]:
# pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("google/gemma-7b", device_map="auto")

model.save_pretrained(r"D:\code\Zoomcamps\llm\LLMBK_HW\WK2\gemma-2b")


To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Trying to resume download...

KeyboardInterrupt



Downloading shards:   0%|                                                                        | 0/4 [00:00<?, ?it/s]C:\Users\ahmed\anaconda3\envs\encode\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ahmed\.cache\huggingface\hub\models--google--gemma-7b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Downloading shards:  25%|███████████████▊                                               | 1/4 [03:56<11:48, 236.29s/it]Error while downloading from https://cdn-lfs-us-1.huggingface.co/repos/31/31/31312cd1e6357f25c1049430e361f8afdb9a455f90748e37d51f3f8e0c6efe61/b9b0278bd4e203c50d4a1b2a29bd6061b19c48abc77f338a4de2f0dd4fba0fac?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00002-of-00004.safetensors%3B+filename%3D%22model-00002-of-00004.safetensors%22%3B&Expires=1720322672&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMDMyMjY3Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzMxLzMxLzMxMzEyY2QxZTYzNTdmMjVjMTA0OTQzMGUzNjFmOGFmZGI5YTQ1NWY5MDc0OGUzN2Q1MWYzZjhlMGM2ZWZlNjEvYjliMDI3OGJkNGUyMDNjNTBkNGExYjJhMjliZDYwNjFiMTljNDhhYmM3N2YzMzhhNGRlMmYwZGQ0ZmJhMGZhYz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=fOC9LgrqHwf9D00x9OnhNa00ytKkdYZUaXu0d4G40Op3a%7EaRAfnSgfkbrsyUz1wVrrAIagWgAvHzIqdy7qYEF-phL1OUiHGU4m7s8itR0f3nMJ0M74supWKA6zOMudPO1I408kAa34Sgh5fduidF4h4dIG0GUEcZiIuCc1s3kYh9-cznjU8GwhBH1BKTIOmgo1VDTQEgyr4q963uTN9aWu9VtXB4ZVXh%7E%7E4tSO44km19i1fMVRvydnMvTdf2gz6iVtRtANYt6W07pHNl6zHk9tXdRdCiVhX35vA%7EczrDwOje4QyK2KIVSqGcqsd7BlTsn5V8PzenKxTJoyA1gDX6dA__&Key-Pair-Id=K24J24Z295AEI9: HTTPSConnectionPool(host='cdn-lfs-us-1.huggingface.co', port=443): Read timed out.
Trying to resume download...
Downloading shards:  25%|███████████████▊                                               | 1/4 [14:19<42:58, 859.50s/it]

KeyboardInterrupt

In [ ]:
#Q5: Docker file
#files were copied from Docker volume. Initially, I tried to download as above but download failed
FROM ollama/ollama:0.1.48

COPY ./models/ /root/.ollama/

In [3]:
#Q6
# model with temperature=0.0 
from openai import OpenAI
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)
def llm_ollama(prompt):


    response = client.chat.completions.create(
        model='gemma:2b',
        temperature=0.0, 
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [1]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm_ollama(prompt)
    return answer

In [10]:
prompt = "What's the formula for energy?"
input_text=llm_ollama(prompt)
print(input_text)
print(len(input_text))

Sure, here's the formula for energy:

**E = K + U**

Where:

* **E** is the energy in joules (J)
* **K** is the kinetic energy in joules (J)
* **U** is the potential energy in joules (J)

**Kinetic energy (K)** is the energy an object possesses when it moves or is in motion. It is calculated as half the product of an object's mass (m) and its velocity (v) squared:

**K = 1/2 * m * v^2**

**Potential energy (U)** is the energy an object possesses when it is in a position or has a specific configuration. It is calculated as the product of an object's mass and the gravitational constant (g) multiplied by the height or distance of the object from a reference point.

**Gravitational potential energy (U)** is given by the formula:

**U = mgh**

Where:

* **m** is the mass of the object in kilograms (kg)
* **g** is the acceleration due to gravity in meters per second squared (m/s^2)
* **h** is the height or distance of the object in meters (m)

The formula for energy can be used to calculate 

In [11]:
prompt = "What's the formula for energy?"
input_text=llm_ollama(prompt)
print(input_text)
print(len(input_text))

Sure, here's the formula for energy:

**E = K + U**

Where:

* **E** is the energy in joules (J)
* **K** is the kinetic energy in joules (J)
* **U** is the potential energy in joules (J)

**Kinetic energy (K)** is the energy an object possesses when it moves or is in motion. It is calculated as half the product of an object's mass (m) and its velocity (v) squared:

**K = 1/2 * m * v^2**

**Potential energy (U)** is the energy an object possesses when it is in a position or has a specific configuration. It is calculated as the product of an object's mass and the gravitational constant (g) multiplied by the height or distance of the object from a reference point.

**Gravitational potential energy (U)** is given by the formula:

**U = mgh**

Where:

* **m** is the mass of the object in kilograms (kg)
* **g** is the acceleration due to gravity in meters per second squared (m/s^2)
* **h** is the height or distance of the object in meters (m)

The formula for energy can be used to calculate 